## Prepare packages

### Basic data science libraries

In [12]:
import numpy as np
import pandas as pd
import pickle
import glob

### Learn-to-rank libraries

In [13]:
# make sure to run this only once
# !conda install -y -c maciejkula -c pytorch spotlight=0.1.5

In [14]:
from spotlight.cross_validation import random_train_test_split
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.evaluation import rmse_score
from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.interactions import Interactions

In [15]:
from spotlight.evaluation import sequence_mrr_score
from spotlight.evaluation import precision_recall_score

### Deep learning libraries

In [16]:
import torch

### Other

In [17]:
import pandas_profiling

In [18]:
from scipy import stats

## Prepare data

In [19]:
import os
print(os.listdir("../input"))

['articles_embeddings.pickle', 'clicks', 'articles_metadata.csv', 'clicks_sample.csv']


### Read articles metadata

In [20]:
articles_meta = pd.read_csv('../input/articles_metadata.csv')
f'There are {len(articles_meta)} rows in the {articles_meta.ndim}-dimensional data frame'

'There are 364047 rows in the 2-dimensional data frame'

### Read articles embeddings

In [21]:
articles_emb = pickle.load(open('../input/articles_embeddings.pickle', 'rb'))
f'There are {len(articles_emb)} rows in the {articles_emb.ndim}-dimensional data frame'

'There are 364047 rows in the 2-dimensional data frame'

In [22]:
articles_emb_df = pd.DataFrame(articles_emb, columns=[f'V{i}' for i in range(0,len(articles_emb[0]))])  # assign colnames basing on the embedding index
# articles_emb_df

### Read clicks data

In [23]:
all_files = glob.glob("../input/clicks/clicks/*")
f"We're going to read {len(all_files)} files"

"We're going to read 385 files"

There are many seperate CSV file holding information on all click events made by registered users. Data is basicly a set of pairs (user_id, article_id) together with metadata like: timestamp of the click, country, etc. The 'click' dataset holds only "positive" labels, meaning that the data don't hold information on the opposite event of 'no click'. 

In [24]:
clicks = pd.concat([pd.read_csv(_) for _ in all_files], ignore_index=True)
f'Successfully read {len(clicks)} rows of the click-data'

'Successfully read 2988181 rows of the click-data'

In [25]:
f'There are {len(set(clicks.user_id.values))} distinct users'

'There are 322897 distinct users'

Join articles (metadata) together with their embeddings to make just one pandas dataframe holding information on all articles. 

In [26]:
articles = pd.concat([articles_meta, articles_emb_df], axis=1)

In [27]:
articles.columns

Index(['article_id', 'category_id', 'created_at_ts', 'publisher_id',
       'words_count', 'V0', 'V1', 'V2', 'V3', 'V4',
       ...
       'V240', 'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248',
       'V249'],
      dtype='object', length=255)

## Exploratory analysis

### First glance at the data

#### Article data

In [28]:
articles.head(10)

,article_id,category_id,created_at_ts,publisher_id,words_count,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,...,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249
0,0,0,1513144419000,0,168,-0.161183,-0.957233,-0.137944,0.050855,0.830055,0.901365,-0.335148,-0.559561,-0.500603,0.165183,0.428434,0.355056,0.874437,-0.528883,0.625487,0.268920,-0.822835,-0.703853,-0.625845,-0.152855,-0.666241,0.043295,0.178638,0.046890,0.594531,-0.183348,0.195107,-0.467640,-0.304807,0.353175,0.278188,0.538623,-0.371209,0.489898,-0.103833,...,0.764381,0.275644,-0.692138,-0.393240,-0.325693,0.163377,-0.154952,-0.701639,0.711825,-0.839063,-0.459145,0.919913,0.621834,-0.640471,0.389934,0.764178,0.164851,-0.747023,0.411034,0.750901,0.860643,0.614755,0.549660,0.334437,-0.388059,-0.703730,-0.567420,0.006478,-0.206590,-0.385272,0.321248,0.313999,0.636412,0.169179,0.540524,-0.813182,0.286870,-0.231686,0.597416,0.409623
1,1,1,1405341936000,0,189,-0.523216,-0.974058,0.738608,0.155234,0.626294,0.485297,-0.715657,-0.897996,-0.359747,0.398246,0.672840,-0.011180,-0.347506,-0.541346,0.458492,0.482316,0.713665,-0.474050,0.405787,0.671431,-0.480856,0.434779,0.468499,0.376424,0.368048,0.192545,0.095572,-0.542234,0.065547,0.363267,-0.162804,0.319617,0.469473,0.578466,-0.803637,...,0.344321,-0.534050,0.728764,0.860110,-0.028166,0.491114,0.531556,0.545350,-0.241544,-0.091119,0.001625,0.950923,-0.367805,-0.215411,-0.131891,0.709081,0.612876,0.814885,0.278680,0.003690,-0.286773,-0.100936,-0.382605,-0.191886,-0.416420,-0.856593,0.537442,0.281693,-0.681829,0.666095,-0.487843,0.823124,0.412688,-0.338654,0.320787,0.588643,-0.594137,0.182828,0.397090,-0.834364
2,2,1,1408667706000,0,250,-0.619619,-0.972960,-0.207360,-0.128861,0.044748,-0.387535,-0.730477,-0.066126,-0.754899,-0.242004,0.670484,-0.280388,-0.557285,-0.084145,0.027782,0.294074,0.362697,-0.368549,0.147960,-0.011751,0.030209,0.106317,0.628013,0.388849,0.615911,-0.445113,0.106028,0.137109,-0.095536,0.342532,0.592646,-0.261791,0.342123,0.704539,-0.433067,...,-0.600840,-0.539323,0.418196,0.028715,0.135008,-0.313483,0.744361,-0.145113,-0.485484,-0.591555,0.030112,0.952736,-0.569376,-0.359680,0.561676,0.381671,-0.189300,0.420950,0.022337,-0.329559,0.232765,0.424368,-0.484598,0.394343,-0.329527,-0.852293,0.634992,-0.532673,-0.469396,0.714991,0.454756,0.473184,0.377866,-0.863887,-0.383365,0.137721,-0.810877,-0.447580,0.805932,-0.285284
3,3,1,1408468313000,0,230,-0.740843,-0.975749,0.391698,0.641738,-0.268645,0.191745,-0.825593,-0.710591,-0.040099,-0.110514,0.591640,-0.245205,-0.788418,-0.345293,-0.756407,0.141115,0.283783,-0.369959,0.374733,0.922633,0.172219,0.253582,0.552108,-0.454131,0.385160,-0.151101,0.598559,-0.614190,-0.258765,0.591907,0.618749,0.414598,0.698184,-0.114862,-0.069056,...,-0.137121,-0.245490,0.170559,0.568089,-0.252295,-0.455726,0.756765,0.626615,-0.005399,-0.472355,0.324625,0.954345,-0.840553,-0.800332,0.111637,-0.501806,0.115986,0.805840,-0.041481,0.320630,-0.450113,-0.378700,0.509616,0.142787,0.149670,-0.896181,0.234389,-0.189831,-0.597612,0.263807,0.271535,0.036040,0.480029,-0.763173,0.022627,0.565165,-0.910286,-0.537838,0.243541,-0.885329
4,4,1,1407071171000,0,162,-0.279052,-0.972315,0.685374,0.113056,0.238315,0.271913,-0.568816,0.341194,-0.600554,-0.125644,0.698668,0.467503,-0.740664,-0.258062,-0.094110,0.256941,0.582071,-0.025041,0.339164,-0.348473,0.413076,0.159945,0.511137,0.608007,0.626225,-0.540690,0.623640,-0.019697,0.027607,-0.236370,0.311671,-0.559831,0.766381,0.412553,-0.663338,...,0.234426,-0.689234,0.751153,0.647424,-0.033631,-0.578617,0.820915,0.370377,-0.528718,-0.536308,-0.088349,0.944439,-0.887679,-0.617357,0.254590,0.084341,0.286366,0.360714,0.369629,-0.452543,-0.292962,-0.530023,-0.237123,0.233008,0.002609,-0.835145,0.494137,0.029535,-0.619638,0.813445,

In [29]:
articles_meta.describe()

,article_id,category_id,created_at_ts,publisher_id,words_count
count,364047.000000,364047.000000,3.640470e+05,364047.0,364047.000000
mean,182023.000000,283.108239,1.474070e+12,0.0,190.897727
std,105091.461061,136.723470,4.293038e+10,0.0,59.502766
min,0.000000,0.000000,1.159356e+12,0.0,0.000000
25%,91011.500000,199.000000,1.444925e+12,0.0,159.000000
50%,182023.000000,301.000000,1.489422e+12,0.0,186.000000
75%,273034.500000,399.000000,1.509891e+12,0.0,218.000000
max,364046.000000,460.000000,1.520943e+12,0.0,6690.000000


In [30]:
articles_emp_profile = pandas_profiling.ProfileReport(articles_emb_df.assign(category_id=articles.category_id).sample(n=3*1000, 
                                                                                                                      weights='category_id', 
                                                                                                                      random_state=42))  # stratified sample per category

In [31]:
articles_emp_profile.get_rejected_variables(threshold=0.9)

['V127', 'V134', 'V150', 'V198', 'V221', 'V235', 'V43', 'category_id']

#### Clean the article data

In [32]:
articles.drop(['publisher_id', 'V127', 'V134', 'V150', 'V198', 'V221', 'V235', 'V43'], axis=1, inplace=True)

In [33]:
articles.columns

Index(['article_id', 'category_id', 'created_at_ts', 'words_count', 'V0', 'V1',
       'V2', 'V3', 'V4', 'V5',
       ...
       'V240', 'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248',
       'V249'],
      dtype='object', length=247)

#### Click data

In [34]:
clicks.head(10)

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,137066,1508106983217640,1508106983000,2,218028,1508107123103,4,1,17,1,25,2
1,137066,1508106983217640,1508106983000,2,214753,1508107153103,4,1,17,1,25,2
2,312478,1508106983626641,1508106983000,2,74719,1508107495538,4,1,17,1,25,4
3,312478,1508106983626641,1508106983000,2,74722,1508107525538,4,1,17,1,25,4
4,266161,1508106984212642,1508106984000,3,331116,1508107023602,4,3,2,1,20,2
5,266161,1508106984212642,1508106984000,3,199474,1508107335633,4,3,2,1,20,2
6,266161,1508106984212642,1508106984000,3,277492,1508107365633,4,3,2,1,20,2
7,33723,1508106984307643,1508106984000,2,284802,1508107248277,4,1,17,1,21,2
8,33723,1508106984307643,1508106984000,2,215613,1508107278277,4,1,17,1,21,2
9,115570,1508106985263644,1508106985000,2,202355,1508107182256,4,4,20,1,26,2


In [35]:
clicks = clicks.infer_objects()

In [36]:
clicks.dtypes

user_id                int64
session_id             int64
session_start          int64
session_size           int64
click_article_id       int64
click_timestamp        int64
click_environment      int64
click_deviceGroup      int64
click_os               int64
click_country          int64
click_region           int64
click_referrer_type    int64
dtype: object

In [37]:
pandas_profiling.ProfileReport(clicks)

Number of variables,12
Number of observations,2988181
Total Missing (%),0.0%
Total size in memory,273.6 MiB
Average record size in memory,96.0 B
Numeric,10
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,2


## Recommender model

In [38]:
small_data = clicks.sample(n=200*1000, weights="click_region", random_state=42)

### Prepare train-test data

In [39]:
dataset = Interactions(user_ids=small_data.user_id.values, 
                       item_ids=small_data.click_article_id.values, 
                       timestamps=small_data.click_timestamp.values)

In [40]:
train, test = random_train_test_split(dataset, test_percentage=0.1)

In [41]:
train, test

(<Interactions dataset (322895 users x 364044 items x 180000 interactions)>,
 <Interactions dataset (322895 users x 364044 items x 20000 interactions)>)

In [42]:
model = ImplicitFactorizationModel(n_iter=4, 
                                   loss='bpr', 
                                   batch_size=32,
                                   use_cuda=torch.cuda.is_available(), 
                                   random_state=np.random.RandomState(42))

In [43]:
model.fit(train, verbose=True)

/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  return torch._C._nn.nll_loss(input, target, weight, size_average, ignore_index, reduce)


Epoch 0: loss 0.144397094260984
Epoch 1: loss 0.013176932133899795
Epoch 2: loss 0.0005321995268265406
Epoch 3: loss 0.00023682927323712244


In [44]:
mrr = mrr_score(model, test)

In [45]:
f'Mean reciprocal rank score is {np.mean(mrr)}'

'Mean reciprocal rank score is 0.029870174154422592'

In [46]:
stats.describe(mrr)

DescribeResult(nobs=18031, minmax=(2.747418800037365e-06, 1.0), mean=0.029870174154422592, variance=0.011798552605954624, skewness=7.174885597789388, kurtosis=57.29601146956058)